In [4]:
# offic venv : py37Torch

In [3]:
!pip install BeautifulSoup4

  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)


In [32]:
import requests
from bs4 import BeautifulSoup

class grant_crawler:

    def url_requests(self,address):
        try:
            req = requests.get(address,timeout=5)
        except:
            return "Timeout error.."
        source = req.text
        return BeautifulSoup(source, 'html.parser')

    def find_target_class_list(self,str_data,target_string):
        index = 0
        table_list = []
        while index > -1:
            index = str_data.find(target_string, index)
            if index > -1:
                table_info = str(str_data[index:str_data.find('>', index)])
                table_list.append(table_info.split("=")[1].split("\"")[1].split(" ")[0])
                index += 1
        return table_list

    def find_target_slicing(self,str_data,target_string):
        index = 0
        while index > -1:
            index = str_data.find(target_string, index)
            if index > -1:
                str_data = str_data[:index]
                index += 1
        return str_data

    def wikipedia_search(self,address):
        ################# 위키피디아 전용 ########################
        url_respond = self.url_requests(address)
        if url_respond == "Timeout error..":
            return "Timeout error.."
        else:
            contents = url_respond

            def section_info(request_instance,selector):
                info_txt = request_instance.select(selector)
                if(len(info_txt)>0):
                    return info_txt[0].text
                else:
                    return ""

            contents_info = contents.select("#mw-content-text > div.mw-parser-output")
            index_info = section_info(contents,"#toc")
            infobox_info = section_info(contents,"#mw-content-text > div.mw-parser-output > table.infobox.vcard")
            ref_info = section_info(contents,"#mw-content-text > div.mw-parser-output > div.reflist")

            filtered_info = contents_info[0].text.replace(index_info,"").replace(infobox_info,"").replace(ref_info,"")

            str_data = str(contents_info[0])
            table_list = self.find_target_class_list(str_data,'<table class=')

            table_result = list(map(lambda x: contents.select("."+str(x)), list(set(table_list))))


            for table_index in range(len(table_result)):
                for table_buffer in range(len(table_result[table_index])):
                    filtered_info = filtered_info.replace(str(table_result[table_index][table_buffer].text),"")

            ## Ref number 삭제
            for N_ref in list(map(lambda x: x.text, contents.select('.reference'))):
                filtered_info = filtered_info.replace(N_ref,"")

            ## Reference 이후 불필요한 부분 삭제
            filtered_info = self.find_target_slicing(filtered_info,"References[edit]")
            filtered_info = filtered_info.replace("[edit]","") ## edit 제거
            filtered_info = filtered_info.split("\n")

            strip_list = list(map(lambda x: x.strip(),filtered_info))

            sentence = ""
            for ingredient in strip_list:
                if(ingredient == ''):
                    pass
                else:
                    ingredient = ingredient.replace("\xa0"," ") ## 이따금 들어오는 공백 널문자는 공백으로 대체함.
                    ## 각 요소 마다 마지막엔 구두점으로 끝나게하자.(문장 구분을 통해 NLP성능 극대화)
                    if(ingredient[-1] == "."):
                        pass
                    else:
                        ingredient += "."
                    sentence += ingredient + " "

            return sentence[:-1]

In [33]:
cr = crawler()

address = "https://stories.starbucks.com/stories/the-starbucks-foundation/"

se_re = cr.url_requests(address)

se_re

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><link href="https://stories.starbucks.com/favicon.ico" rel="shortcut icon"/><meta content="width=device-width,initial-scale=1,shrink-to-fit=no" name="viewport"/><meta content="#000000" name="theme-color"/><link href="https://stories.starbucks.com/manifest.json" rel="manifest"/><title>Starbucks</title><link href="https://stories.starbucks.com/static/css/130.6cccecdc.chunk.css" rel="stylesheet"/><link href="https://stories.starbucks.com/static/css/main.a8580acc.chunk.css" rel="stylesheet"/></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div class="sb-app-wrap" id="root"></div><div id="modal-root"></div><script name="x-state"></script><script>!function(e,t,a,n,g){e[n]=e[n]||[],e[n].push({"gtm.start":(new Date).getTime(),event:"gtm.js"});var m=t.getElementsByTagName(a)[0],r=t.createElement(a);r.async=!0,r.src="https://www.googletagmanager.com/gtm.js?id=GTM-P93LDX",m.parentNode.insertBefore(r,m)}(w